In [23]:
import gspread
import pandas as pd
import matplotlib.pyplot as plt
from google.oauth2.service_account import Credentials

In [3]:
SERVICE_ACCOUNT_FILE = '../../key/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

sheet = client.open("[3] AI QC Inbound CRM Review 语音智能质检打标复审 ").worksheet("Sampling")
data = sheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32030 entries, 0 to 32029
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   NO                                32030 non-null  object
 1   Tanggal Sampling                  32030 non-null  object
 2   Agent Sampling                    32030 non-null  object
 3   Tanggal Pengerjaan                32030 non-null  object
 4   Checker                           32030 non-null  object
 5   ASI/AFI                           32030 non-null  object
 6   Waktu Inbound                     32030 non-null  object
 7   Nomor Inbound                     32030 non-null  object
 8   Nama Agent                        32030 non-null  object
 9   Call ID                           32030 non-null  object
 10  Durasi Bicara                     32030 non-null  object
 11  Detik                             32030 non-null  object
 12  Total Tiket Riskan

In [5]:
# Data Cleaning
df_clean = df.copy()

# Bersihkan nama kolom
df_clean.columns = [
    col.strip() if col.strip() else 'Catatan Tambahan'
    for col in df_clean.columns
]
df_clean.columns = df_clean.columns.str.strip()

# Penyesuaian nama checker
df_clean['Checker'] = df_clean['Checker'].replace({
    "Aul": "Aulia",
    "Iman": "Irman",
})

# Konversi kolom tanggal
date_cols = ['Tanggal Sampling','Tanggal Pengerjaan', 'Waktu Inbound']
for col in date_cols:
    df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Konversi kolom numerik
numeric_cols = ['NO', 'Count Hasil ASR', 'Count Hasil Pemeriksaan Kualitas', 'Count Efektif', 'Count Kejelasan Suara', 'Count Suara Lain', 'Count Kelengkapan Rekaman', 'Count Revisi Text']
for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].replace('No Data', 0)
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0).astype(int)

# Bersihkan kolom teks
text_cols = df_clean.select_dtypes(include='object').columns
df_clean[text_cols] = df_clean[text_cols].apply(lambda x: x.astype(str).str.strip())

# Ganti string kosong dengan 'No Data'
df_clean.replace('', 'No Data', inplace=True)

# Konversi kolom kategori
categorical_cols = [
    'Checker', 'ASI/AFI', 'Status',
    'Efektif', 'Kejelasan Suara', 'Suara Lain',
    'Kelengkapan Rekaman', 'Sampling user side', 
    'Hasil ASR', 'Hasil Pemeriksaan Kualitas', 'Agent Sampling'
]
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('category')

# Drop kolom tidak diperlukan 
if 'Status' in df_clean.columns:
    df_clean.drop(columns='Status', inplace=True)

# Info dataframe
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32030 entries, 0 to 32029
Data columns (total 32 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   NO                                32030 non-null  int64         
 1   Tanggal Sampling                  32030 non-null  datetime64[ns]
 2   Agent Sampling                    32030 non-null  category      
 3   Tanggal Pengerjaan                32030 non-null  datetime64[ns]
 4   Checker                           32030 non-null  category      
 5   ASI/AFI                           32030 non-null  category      
 6   Waktu Inbound                     32030 non-null  datetime64[ns]
 7   Nomor Inbound                     32030 non-null  object        
 8   Nama Agent                        32030 non-null  object        
 9   Call ID                           32030 non-null  object        
 10  Durasi Bicara                     32030 non-nu

In [6]:
df_clean

,NO,Tanggal Sampling,Agent Sampling,Tanggal Pengerjaan,Checker,ASI/AFI,Waktu Inbound,Nomor Inbound,Nama Agent,Call ID,...,Kelengkapan Rekaman,Revisi Teks,Red Label,Count Hasil ASR,Count Hasil Pemeriksaan Kualitas,Count Efektif,Count Kejelasan Suara,Count Suara Lain,Count Kelengkapan Rekaman,Count Revisi Text
0,1,2025-03-11,Aulia,2025-03-03,Azer,AFI,2025-02-28 10:49:24,0812****5797,Naella Absoni,770789e745fb4182853763cd8d1b6a40,...,0 Utuh,No Data,No Data,0,0,0,0,0,0,0
1,2,2025-03-11,Aulia,2025-03-03,Azer,AFI,2025-02-28 10:49:24,0812****5797,Naella Absoni,770789e745fb4182853763cd8d1b6a40,...,0 Utuh,No Data,No Data,0,0,0,0,0,0,0
2,3,2025-03-11,Aulia,2025-03-03,Azer,AFI,2025-02-28 10:49:24,0812****5797,Naella Absoni,770789e745fb4182853763cd8d1b6a40,...,1 Tidak Utuh,No Data,No Data,0,0,0,0,0,0,0
3,4,2025-03-11,Aulia,2025-03-03,Azer,AFI,2025-02-28 10:49:24,0812****5797,Naella Absoni,770789e745fb4182853763cd8d1b6a40,...,1 Tidak Utuh,No Data,No Data,0,0,0,0,0,0,0
4,5,2025-03-11,Aulia,2025-03-03,Azer,AFI,2025-02-28 10:49:24,0812****5797,Naella Absoni,770789e745fb4182853763cd8d1b6a40,...,0 Utuh,No Data,No Data,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32025,32025,2025-11-04,Neneng,2025-10-30,Azer,AFI,2025-10-29 19:11:28,0857****1505,Nurul Wahyu Ramadhani,86a2dfbb08e241219ee90d938c6aa155,...,0 Utuh,No Data,No Data,0,0,0,0,0,0,0
32026,32026,2025-11-04,Neneng,2025-10-30,Azer,AFI,2025-10-29 19:11:28,0857****1505,Nurul Wahyu Ramadhani,86a2dfbb08e241219ee90d938c6aa155,...,0 Utuh,No Data,No Data,0,0,0,0,0,0,0
32027,32027,2025-11-04,Neneng,2025-10-30,Azer,AFI,2025-10-29 19:11:28,0857****1505,Nurul Wahyu Ramadhani,86a2dfbb08e241219ee90d938c6aa155,...,0 Utuh,No Data,No Data,0,0,0,0,0,0,0
32028,32028,2025-11-04,Neneng,2025-10-30,Azer,AFI,2025-10-29 19:11:28,0857****1505,Nurul Wahyu Ramadhani,86a2dfbb08e241219ee90d938c6aa155,...,1 Tidak Utuh,No Data,No Data,0,0,0,0,0,0,0


In [32]:
# count_cols = [
#     'Count Hasil ASR',
#     'Count Hasil Pemeriksaan Kualitas',
#     'Count Efektif',
#     'Count Kejelasan Suara',
#     'Count Kelengkapan Rekaman',
#     'Count Revisi Text'
# ]

# # Filter sesuai tanggal
# df_filtered = df_clean.loc[
#     df_clean['Tanggal Sampling'].between('2025-10-27', '2025-10-31', inclusive='both'),
#     ["Tanggal Sampling"] + count_cols
# ]

# # Hitung total tiap kolom dan urutkan dari besar ke kecil
# count_sum = df_filtered[count_cols].sum().sort_values(ascending=False)

# # Plot bar chart
# plt.figure(figsize=(10,6))
# bars = plt.bar(count_sum.index, count_sum.values, color='steelblue')

# # Tambahkan angka di atas setiap bar
# for bar in bars:
#     plt.text(
#         bar.get_x() + bar.get_width()/2,
#         bar.get_height(),
#         f'{int(bar.get_height()):,}',   # format angka
#         ha='center', va='bottom', fontsize=10
#     )

# plt.title("Total Count per Category (2025-10-27 s.d. 2025-10-31)")
# plt.ylabel("Total Count")
# plt.xticks(rotation=45, ha='right')
# plt.grid(axis='y', linestyle='--', alpha=0.4)
# plt.tight_layout()
# plt.show()


In [ ]:
df_clean.to_csv("../../dataset_qc/kalib_sampling.csv")